###### 작성자 : 손해원
- 주 기능 : 구글지도에서 키워드로 검색 후 해당 장소에 대한 리뷰를 수집합니다.
- 수집 가능 항목 : 장소 이름, 페이지url, 리뷰작성자, 리뷰날짜, 리뷰내용, 별점

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pandas as pd
import time

In [2]:
driver_path = 'C:/Users/seoyh/Desktop/PythonWorkspace/webdriver/chromedriver' # 크롬드라이버 경로 설정
keyword = '김천 숙박'   ## 검색어 설정 
file_path = 'C:/Users/seoyh/Desktop/PythonWorkspace/Jupyter_worksapce/Data/2022-01-11'    ### 저장할 경로 
file_name = 'google_김천숙박'      ### 저장할 이름 

In [3]:
url = f'https://www.google.com/maps/search/{keyword}' 

In [4]:
driver = webdriver.Chrome(driver_path)
driver.get(url)

C:\Users\seoyh\AppData\Local\Temp/ipykernel_16888/3304505801.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


##### url 수집

In [5]:
url_list = []
j = 0

for x in range(20):
    for i in range(10): # 페이지 스크롤 다운 기능 
        driver.find_element_by_class_name('a4gq8e-aVTXAb-haAclf-jRmmHf-hSRGPd').send_keys(Keys.PAGE_DOWN)

    upso_list = driver.find_elements_by_class_name('a4gq8e-aVTXAb-haAclf-jRmmHf-hSRGPd') # 식당 리스트
    try:
        for j in range(len(upso_list)): # url 스크랩 
            time.sleep(2)
            detail_url = upso_list[j].get_attribute('href')
            url_list.append(detail_url)
            j += 1
    except:
        continue
        
    try:
        driver.find_element_by_css_selector('#ppdPk-Ej1Yeb-LgbsSe-tJiF1e > img').click() # 다음 페이지 클릭 
    except:
        break
        
### 검색 내역이 끝나면 종료됨

C:\Users\seoyh\AppData\Local\Temp/ipykernel_16888/3208370182.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name('a4gq8e-aVTXAb-haAclf-jRmmHf-hSRGPd').send_keys(Keys.PAGE_DOWN)
C:\Users\seoyh\AppData\Local\Temp/ipykernel_16888/3208370182.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  upso_list = driver.find_elements_by_class_name('a4gq8e-aVTXAb-haAclf-jRmmHf-hSRGPd') # 식당 리스트
C:\Users\seoyh\AppData\Local\Temp/ipykernel_16888/3208370182.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#ppdPk-Ej1Yeb-LgbsSe-tJiF1e > img').click() # 다음 페이지 클릭


In [6]:
print('수집된 url 개수:',len(url_list),'개')

수집된 url 개수: 209 개


###### 리뷰 스크랩

In [7]:
review_list = []

i = 0
x = 0
j = 1
for x in range(len(url_list)):
    
    # url로 이동 
    driver.get(url_list[x])
    time.sleep(1)
    x += 1
    
    # 식당 이름
    restaurant = driver.find_element_by_class_name('x3AX1-LfntMc-header-title-title.gm2-headline-5').text
    
    try:
        # 리뷰 더보기 클릭 
        driver.find_element_by_class_name('Yr7JMd-pane-hSRGPd').send_keys(Keys.ENTER)
#         driver.find_element_by_css_selector('#pane > div > div.Yr7JMd-pane-content.cYB2Ge-oHo7ed > div > div > div.x3AX1-LfntMc-header-title > div.x3AX1-LfntMc-header-title-ma6Yeb-haAclf > div.x3AX1-LfntMc-header-title-ij8cu > div.x3AX1-LfntMc-header-title-ij8cu-haAclf > div > div.gm2-body-2.h0ySl-wcwwM-RWgCYc > span:nth-child(3) > span > span > span.OAO0-ZEhYpd-vJ7A6b.OAO0-ZEhYpd-vJ7A6b-qnnXGd > span:nth-child(1) > button').click()
        time.sleep(1)

        try:

            for i in range(500): # 리뷰 500개까지 스크랩
                review_dict = {}
                
                name = driver.find_elements_by_class_name('ODSEW-ShBeI-title')[i].text # 작성자 
#                 date = driver.find_elements_by_class_name('ODSEW-ShBeI-RgZmSc-date')[i].text # 작성일
                date = driver.find_elements_by_class_name('ODSEW-ShBeI-RgZmSc-date-J42Xof-Hjleke')[i].text.replace('\nGoogle\n에 게시됨','')
                text = driver.find_elements_by_class_name('ODSEW-ShBeI-text')[i].text # 내용
                rating = int(driver.find_elements_by_class_name('ODSEW-ShBeI-RGxYjb-wcwwM')[i].text.replace('/5',''))
                
                review_dict['dataSource'] = 'GOOGLEMAPS'
                review_dict['searchKeyword'] = f'{keyword}' 
                review_dict['reviewTarget'] = restaurant
                review_dict['author'] = name
                review_dict['reviewDate'] = date
                review_dict['reviewText'] = text.replace('\n',' ')
                review_dict['rating'] = rating
                review_dict['reviewUrl'] = url_list[x]
                
                print(review_dict)
                review_list.append(review_dict)
                
                i += 1

                # 스크롤 다운 
                driver.find_element_by_class_name('siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc').send_keys(Keys.PAGE_DOWN)
        except:
            print('ok')
            continue
    except:
        continue



C:\Users\seoyh\AppData\Local\Temp/ipykernel_16888/2153921590.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  restaurant = driver.find_element_by_class_name('x3AX1-LfntMc-header-title-title.gm2-headline-5').text


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".x3AX1-LfntMc-header-title-title.gm2-headline-5"}
  (Session info: chrome=97.0.4692.71)
Stacktrace:
Backtrace:
	Ordinal0 [0x0043FDC3+2555331]
	Ordinal0 [0x003D77F1+2127857]
	Ordinal0 [0x002D2E08+1060360]
	Ordinal0 [0x002FE49E+1238174]
	Ordinal0 [0x002FE69B+1238683]
	Ordinal0 [0x00329252+1413714]
	Ordinal0 [0x00317B54+1342292]
	Ordinal0 [0x003275FA+1406458]
	Ordinal0 [0x00317976+1341814]
	Ordinal0 [0x002F36B6+1193654]
	Ordinal0 [0x002F4546+1197382]
	GetHandleVerifier [0x005D9622+1619522]
	GetHandleVerifier [0x0068882C+2336844]
	GetHandleVerifier [0x004D23E1+541697]
	GetHandleVerifier [0x004D1443+537699]
	Ordinal0 [0x003DD18E+2150798]
	Ordinal0 [0x003E1518+2168088]
	Ordinal0 [0x003E1660+2168416]
	Ordinal0 [0x003EB330+2208560]
	BaseThreadInitThunk [0x75AAFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77447A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77447A6E+238]


##### DataFrame

In [ ]:
review_df = pd.DataFrame(review_list)
review_df

##### To csv file

In [ ]:
review_df.to_csv(f'{file_path}{file_name}.csv', index=True)